In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


# 다음 사이트의 연령대별 랭킹 뉴스 정보 수집하기
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt 
import random
import os

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 이 크롤러는 다음 사이트 랭킹 뉴스중 연령별 인기 기사 정보를 수집합니다")
print("=" *80)
print("\n")

st_date = input('1.조회 시작일을 입력하세요(예:20160101) :  ')
en_date = input('2.조회 종료일을 입력하세요(예:20161231) :  ')


s_year=st_date[0:4]
e_year= en_date[0:4]


#year = input("1.크롤링을 원하는 년도를 입력하세요:")
url = 'https://media.daum.net/ranking/age/?regDate='  

f_dir=input('3.파일이 저장될 경로만 쓰세요;기본값:c:\\temp\\): ')
if f_dir=='' :
    f_dir="c:\\temp\\"
    
print("\n")
print("요청하신 정보를 수집중이니 잠시만 기다려 주세요~~^^")
print("\n")

#Step 3. 결과를 저장할 파일과 폴더명을 설정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
  
os.makedirs(f_dir+'다음뉴스_연령별_20대이하여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_20대이하남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_30대여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_30대남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_40대여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_40대남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_50대이상여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_연령별_50대이상남성_'+s_year+'-'+s)

ff_20_w_name=f_dir+'다음뉴스_연령별_20대이하여성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_20대이하여성_'+s_year+'-'+s+'.txt'
ff_20_m_name=f_dir+'다음뉴스_연령별_20대이하남성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_20대이하남성_'+s_year+'-'+s+'.txt'
ff_30_w_name=f_dir+'다음뉴스_연령별_30대여성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_30대여성_'+s_year+'-'+s+'.txt'
ff_30_m_name=f_dir+'다음뉴스_연령별_30대남성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_30대남성_'+s_year+'-'+s+'.txt'
ff_40_w_name=f_dir+'다음뉴스_연령별_40대여성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_40대여성_'+s_year+'-'+s+'.txt'
ff_40_m_name=f_dir+'다음뉴스_연령별_40대남성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_40대남성_'+s_year+'-'+s+'.txt'
ff_50_w_name=f_dir+'다음뉴스_연령별_50대이상여성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_50대이상여성_'+s_year+'-'+s+'.txt'
ff_50_m_name=f_dir+'다음뉴스_연령별_50대이상남성_'+s_year+'-'+s+'\\'+'다음뉴스_연령별_50대이상남성_'+s_year+'-'+s+'.txt'

#Step 4. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

#Step 5. 날짜 관련 작업하기


mon=["01","02","03","04","05","06","07","08","09","10","11","12"]
i=0

start_date2=[]
end_date2=[]

for s in range(int(s_year),int(e_year)+1):
    for i in range(0,len(mon)) :
      if mon[i] =="02" :
        sdate=str(s)+mon[i]+'01'
        start_date2.append(sdate)

        edate=str(s)+mon[i]+'28'
        end_date=edate
        end_date2.append(end_date)

      elif mon[i] == "04" or mon[i]=="06" or mon[i]=="09" or mon[i]=="11" :
        sdate=str(s)+mon[i]+'01'
        start_date2.append(sdate)

        edate=str(s)+mon[i]+'30'
        end_date=edate
        end_date2.append(end_date)
      else :
        sdate=str(s)+mon[i]+'01'
        start_date2.append(sdate)

        edate=str(s)+mon[i]+'31'
        end_date=edate
        end_date2.append(end_date)
    
    

#print(start_date2)
#print(end_date2)

#Step 6. 각 연령대별 자료 수집하기
count_20_w = 0
count_20_m = 0

count_30_w = 0
count_30_m = 0

count_40_w = 0
count_40_m = 0

count_50_w = 0
count_50_m = 0

for x in range(0,len(end_date2)) :
    for i in range(int(start_date2[x]),int(end_date2[x])) :
        full_url = url+str(i)

        c_year=str(i)[0:4]
        c_mon=str(i)[4:6]
        c_day=str(i)[6:]
        
  
        driver.get(full_url)
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    
        article_result = soup.select('div.rank_news')
        print("%s년 %s월 %s일의 뉴스 수집 중 ===================================" %(c_year,c_mon,c_day))

        for li in article_result:

            #20대 이하 여성 뉴스 모으기
            f1 = open(ff_20_w_name, 'a',encoding='UTF-8')
                
            w20_news = li.find('div',class_='item_age item_20s').find('div','rank_female').find('ol')
            print("*%s년 %s월 %s일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no = 1
            for a in range(0,len(w20_news)-1) :
                
                
                txt1=w20_news.find_all('a')
                try :
                    title1 = txt1[a].get_text()
                    print(no,':',title1)
                except IndexError :
                    continue
            
                count_20_w += 1
                
                try :     
                    f1.write(str(no)+':'+title1 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f1.write("\n")    
            f1.close( )
        
            print("w20=======================================================")
            print("\n")

            #20대 이하 남성 뉴스 모으기
            f2 = open(ff_20_m_name, 'a',encoding='UTF-8')
        
            m20_news = li.find('div',class_='item_age item_20s').find('div','rank_male').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no = 1
            for b in range(0,len(m20_news)-1) :
                
                txt2 = m20_news.find_all('a')
                try :
                    title2 = txt2[b].get_text() 
                    print(no,':',title2)
                except IndexError :
                    continue
            
                count_20_m += 1
                try :     
                    f2.write( str(no)+':'+title2 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f2.write("\n")    
            f2.close( )
            print("m20====================================================")
            print("\n")

            #30대 여성 뉴스 모으기
            f3 = open(ff_30_w_name, 'a',encoding='UTF-8')
        
            w30_news = li.find('div',class_='item_age item_30s').find('div','rank_female').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no = 1
            for c in range(0,len(w30_news)-1) :
                
                txt3 = w30_news.find_all('a')
                try :
                    title3 = txt3[c].get_text() 
                    print(no,':',title3)
                except IndexError :
                    continue
                count_30_w += 1
                try :     
                    f3.write( str(no)+':'+title3 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f3.write("\n")    
            f3.close( )
                               
            print("w30====================================================")
            print("\n")

            #30대 남성 뉴스 모으기
            f4 = open(ff_30_m_name, 'a',encoding='UTF-8')
        
            m30_news = li.find('div',class_='item_age item_30s').find('div','rank_male').find('ol')
    
            print("*%s년 %s월 %s일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no = 1
            for d in range(0,len(m30_news)-1) :
                
                txt4 = m30_news.find_all('a')
                try :
                    title4 = txt4[d].get_text() 
                    print(no,':',title4)
                except IndexError :
                    continue
                count_30_m += 1
                try :     
                    f4.write(str(no)+':'+ title4 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f4.write("\n")
            f4.close( )
        
            print("m30====================================================")
            print("\n")

            #40대 여성 뉴스 모으기
            f5 = open(ff_40_w_name, 'a',encoding='UTF-8')
        
            w40_news = li.find('div',class_='item_age item_40s').find('div','rank_female').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 40대 여성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for e in range(0,len(w40_news)-1) :
                
                txt5 = w40_news.find_all('a')
                try :
                    title5 = txt5[e].get_text()
                    print(no,':',title5)
                except IndexError :
                    continue
            
                count_40_w += 1
                try :     
                    f5.write( str(no)+':'+title5 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f5.write("\n")
            f5.close( )
        
            print("w40====================================================")
            print("\n")

            #40대 남성 뉴스 모으기
            f6 = open(ff_40_m_name, 'a',encoding='UTF-8')
            m40_news = li.find('div',class_='item_age item_40s').find('div','rank_male').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 40대 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for f in range(0,len(m40_news)-1) :
                
                txt6 = m40_news.find_all('a')
                try :
                    title6 = txt6[f].get_text() 
                    print(no,':',title6)
                except IndexError :
                    continue
                count_40_m += 1
                try :     
                    f6.write( str(no)+':'+title6 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f6.write("\n")
            f6.close( )
        
            print("m40====================================================")
            print("\n")

            #50대 이상 여성 뉴스 모으기
            f7 = open(ff_50_w_name, 'a',encoding='UTF-8')
        
            w50_news = li.find('div',class_='item_age item_50s').find('div','rank_female').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 50대 여성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for g in range(0,len(w50_news)-1) :
             
                txt7 = w50_news.find_all('a')
                try :
                    title7 =  txt7[g].get_text()
                    print(no,':',title7)
                except IndexError :
                    continue
            
                count_50_w += 1
                try :     
                    f7.write(str(no)+':'+ title7 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f7.write("\n")
                
            f7.close( )
            print("w50====================================================")
            print("\n")
        
            #50대 이상 남성 뉴스 모으기
            f8 = open(ff_50_m_name, 'a',encoding='UTF-8')
        
            m50_news = li.find('div',class_='item_age item_50s').find('div','rank_male').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 50대 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for h in range(0,len(m50_news)-1) :
            
                txt8 = m50_news.find_all('a')
                try :
                    title8 =  txt8[h].get_text()
                    print(no,':',title8)
                except IndexError :
                    continue
            
                count_50_m += 1
                try :     
                    f8.write(str(no)+':'+ title8 + "\n")
                    
                except AttributeError:
                    continue
                no += 1
            f8.write("\n")
            f8.close( )
        
            print("m50====================================================")
            print("\n")
        
            time.sleep(0.4)
                               
        if str(i) == en_date :
            break
            
    if str(i) == en_date :
            break

# Step 7.작업결과 요약 정보 보여주기

e_time = time.time( )
t_time = e_time - s_time            
print("=" *80)            
print("1.20대 이하 여성 : 총 %s 건  완료" %count_20_w)
print("2.20대 이하 남성 : 총 %s 건  완료" %count_20_m)
print("3.30대 여성      : 총 %s 건  완료" %count_30_w)
print("4.30대 남성      : 총 %s 건  완료" %count_30_m)
print("5.40대 여성      : 총 %s 건  완료" %count_40_w)
print("6.40대 남성      : 총 %s 건  완료" %count_40_m)
print("7.50대 이상 여성 : 총 %s 건  완료" %count_50_w)
print("8.50대 이상 남성 : 총 %s 건  완료" %count_50_m)
print("=" *80 )                                            
print("\n")
print("파일이 저장된 경로는 아래와 같습니다")
print("=" *80)
print("1.20대 이하 여성: %s"  %ff_20_w_name)
print("2.20대 이하 남성: %s"  %ff_20_m_name)
print("3.30대      여성: %s"  %ff_30_w_name)
print("4.30대      남성: %s"  %ff_30_m_name)
print("5.40대      여성: %s"  %ff_40_w_name)
print("6.40대      남성: %s"  %ff_40_m_name)
print("7.50대 이상 여성: %s"  %ff_50_w_name)
print("8.50대 이상 남성: %s"  %ff_50_m_name)
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))

driver.close( )


# In[ ]:

 이 크롤러는 다음 사이트 랭킹 뉴스중 연령별 인기 기사 정보를 수집합니다


1.조회 시작일을 입력하세요(예:20160101) :  2019-01-01
2.조회 종료일을 입력하세요(예:20161231) :  2019-01-10
3.파일이 저장될 경로만 쓰세요;(예:c:\temp\): c:\temp\


요청하신 정보를 수집중이니 잠시만 기다려 주세요~~^^


2019년 01월 01일의 뉴스 수집 중 ===================================
*2019년 01월 01일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : '비닐봉투 사용 금지' 첫날..시민들 "이제 안 써야죠"
2 : 지난해 12월 마지막 밤 울릉도 밤하늘에 무슨 일이 ?
3 : [월드피플+] 77년 만에 배달된 러브레터..글에 담긴 가슴아픈 사연
4 : 엑소 카이♥블랙핑크 제니 열애설..소속사 묵묵부답
5 : 효린, KBS연기대상 축하무대 의상 '선정성 논란'
w20=======================================================


*2019년 01월 01일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : [우주를 보다] 화성 북극권에서 대형 얼음 크레이터 발견
2 : 文대통령, 새해 남산에서 2018 의인들과 해맞이(종합)
3 : "밤에 문 열어봤자 손해".. '편의점=24시간' 공식 깨지나 [뉴스+]
4 : '블랙 수트' 입고 집무실 소파에서..파격 연출
5 : "통행세 내기 싫어", 탈(脫) 애플·구글 본격 확산되나
m20====================================================


*2019년 01월 01일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 맨발 운동 30분의 마법.. 아이들 뇌가 깨어났다
2 : 0시에 새해 첫아기 힘찬 울음소리..자연분만 2.93kg 건강한 딸
3 : 배우 이영애, 폐업위기 처한 제일병원 인수 나선다
4 : 달리던 

2019년 01월 05일의 뉴스 수집 중 ===================================
*2019년 01월 05일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : "산천어축제가 뭐길래"..첫날 축구장 70배 얼음벌판 인파 가득
2 : 보호자 떠나고 집안에 방치됐다 구조된 비글
3 : 서울대병원, 방검조끼-삼단봉-전기충격기 '무장 요원' 배치
4 : 출산율 줄었다고? 2030 혼인가구는 늘었다
5 : "4명 목숨 구하고, 저 이제 은퇴해요 멍멍!"
w20=======================================================


*2019년 01월 05일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : 유시민 알릴레오, 첫날부터 흥행몰이..유튜브 구독자 21만 돌파
2 : 송악산에 뚫린 동굴.. 누가 제주를 아프게 했나
3 : 봉우리마다 궁전 하나..꼭대기엔 반란 황제의 미니 자금성
4 : 중국 창어4호서 분리된 '옥토끼', 달 뒷면 순조롭게 탐사
5 : '日 잘못' 동영상 '160만' 조회..국제 여론전
m20====================================================


*2019년 01월 05일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 네 아이 아빠 '82년생 부부' 이종섭씨의 눈물콧물 현실육아 [커버스토리]
2 : "우리도 먹고살아야죠" 의류수거함 관리자 수익은 어떻게?
3 : 방탈출게임방 화재참사..폴란드서 10대 소녀 5명 사망
4 : 일제식 '국민학교'는 '초등학교' 됐는데..유치원은?
5 : 임대료·인건비 부담에 사라지는 '저가 커피'..이디야 이어 더벤티도 가격 인상
w30====================================================


*2019년 01월 05일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : "목적지까지 데려다 주면 됐지"..택시업계, 서비스 의지 無

2019년 01월 09일의 뉴스 수집 중 ===================================
*2019년 01월 09일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : 8톤트럭 언덕서 100m 굴러 시내버스 덮쳐..9명 사상(종합)
2 : 광화문광장 인근 도로서 택시 화재.."기사 분신시도 추정"(종합2보)
3 : '47년 금녀의 벽' 깨고.. 서울 첫 女소방서장
4 : 심석희, 뒤늦게 조재범 성폭행 고백한 이유.."팬 편지 때문"
5 : 태어나니 '낀 세대' 1992년생을 구하라 [다시 쓰는 인구론]
w20=======================================================


*2019년 01월 09일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : [CES 2019]은밀하게 공개한 폴더블폰..눈치 작전 치열해진 CES
2 : 부산 도심에 나타난 대형 멧돼지 순찰차로 들이받아 생포
3 : 트랙터로 폐기되는 가을 배추
4 : "'주차전쟁터' 강남 한복판서 경찰이"..경찰차 불법주차 논란
5 : 새해 첫날, 눈앞에서 벌어진 멸종
m20====================================================


*2019년 01월 09일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 심석희 "라커룸에서도 성폭행"..경찰, 조재범 코치 휴대폰 등 압수 수사
2 : 심석희, 부친과 함께 정신과 치료..고소장엔 피해 10여 건 담겨
3 : 말레이 전 국왕과 결혼 러시아 모델, 임신..올해 출산 전망
4 : [단독] "빙상계 성폭력 피해 선수 더 있다"
5 : [나는 너의 야동이 아니다] 미녀와의 짜릿한 채팅.. 눈치 챘을 땐 나는 이미 노예였다
w30====================================================


*2019년 01월 09일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : "15일 일해도 편의점

2019년 01월 13일의 뉴스 수집 중 ===================================
*2019년 01월 13일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : 여주서 부부싸움 중 불..일가족 5명 부상
2 : 망한 '청년몰' 지척에 또 '청년몰'..690억원 투입해 30% 폐업
3 : 소유·소비·연애 담쌓고 미래 걱정하는 일본 청년들 [이동준의 일본은 지금]
4 : '청량리 집창촌' 폐상가 옥상서 쇠사슬 시위..철거보상 요구
5 : 프랑스 파리 빵집서 가스폭발..소방관 2명 등 3명 사망 부상자 47명
w20=======================================================


*2019년 01월 13일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : 가장 오래 하늘 머무른 비행선 새 모델로 교체하려고 지상에
2 : 파주시 "군사적 효용성 낮은 대전차 방호벽 철거 추진"
3 : 3개월 전에도 리만 가설 증명했는데 수학자 아티야 89세로 별세
4 : 美NBC 앵커 메긴 켈리, 330억 받고 NBC와 결별
5 : 日 서쪽 해상서 '北어선' 표류.."일본의 구조는 희망 안해"
m20====================================================


*2019년 01월 13일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : [알쓸신세] 돈만 주면 아이 낳는다..임신 하청 '구글 베이비'
2 : "키우던 개 팔아요"..반려동물 중고사이트 거래 논란
3 : 캄보디아서 봉사 중 숨진 건양대생 2명, 국내서 부검 통해 정확한 사인 규명키로
4 : 전 세계가 푹 빠진 동요..빌보드 차트 오른 '아기상어 송'
5 : 이마트몰, 14일부터 닷새간 100원 특가 선보인다
w30====================================================


*2019년 01월 13일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====

2019년 01월 17일의 뉴스 수집 중 ===================================
*2019년 01월 17일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : 15년째 의식 못 찾은 경찰관 '아빠' 대신 막내 딸 졸업식 찾은 동료들
2 : 시칠리아 마을 "단돈 1유로에 집 드려요"
3 : "성폭력 피해자, 싸워서 이긴 사람들로 기억하자"
4 : 매년 날아오는 적십자 용지.. 내 주소·이름 어떻게 아는 걸까
5 : [IF] [사이언스 샷] 중세에도 여성들이 예술에서 중요한 역할
w20=======================================================


*2019년 01월 17일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : 해병대 창설 70주년 퍼포먼스
2 : 신호대기 피하려다 6중 추돌 11명 부상..운전자 입건
3 : [단독] '알츠하이머 골프' 전두환.."스코어 암산까지 했다"
4 : 제주서 전기렌터카 식당으로 돌진..운전자 등 3명 중상
5 : 입 냄새 유발하는 노란 알갱이, 편도결석.. 혹시 나도?
m20====================================================


*2019년 01월 17일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 미세먼지 마스크의 불편한 진실
2 : [여기는 인도] 사람들이 던진 화염병 피하는 코끼리 모자 포착
3 : [단독]"박소연 케어대표, 훔친 개로 축구팀 후원금 받아"
4 : "눈 예쁘게 보이고 싶어도..'미세먼지' 심한 날엔 참아주세요"
5 : [단독] 3살 아들 숨지도록 내버려둔 아빠..재산 절반 상속받는다
w30====================================================


*2019년 01월 17일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : '산소 1캔 13500원 vs 산소 한 봉지 1600원' 미세먼지 대처도 양극화?


...

2019년 03월 16일의 뉴스 수집 중 ===================================
*2019년 03월 16일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : 가수 강남 이상화 공개연애 인정? '들켜도 상관없어'
2 : [단독]AI가 만든 감쪽같은 '정준영-아이돌 몰카' 확산
3 : '박한별 남편' 유인석, 유리홀딩스 대표 사임
4 : [아무튼, 주말] 요즘 대학생들은 선후배가 서로 "○○씨"로 부른다
5 : 여학생들 日교사 턱 앞에 두 손 들이대고 "대한독립 만세"
w20=======================================================


*2019년 03월 16일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : 세계문화유산 '앙코르와트' 둘러 본 문 대통령
2 : "변기에 휴지 버려" vs "버리지 마" 공중 화장실에 '휴지통'을 둘 수밖에 없는 이유 [김기자의 현장+]
3 : 역할 못하는 '시선 유도봉'..깨지고·뽑히고
4 : "우파 정권이 한국경제를 벼랑 아래로 떨어뜨렸다" 말했다면
5 : [단독] 주차 요금 징수하던 장애인 무차별 폭행
m20====================================================


*2019년 03월 16일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 정준영이 유인석 대표에게 받은 크리스마스 선물은 '여자(?)'
2 : [단독풀영상] 마포대교 투신, 그녀의 몸에서 스크루에 베인 상처가?
3 : '승리 라면' 측 유리홀딩스와 관계 정리하겠다더니..신임 대표 논란
4 : 미세먼지, 잠자는 '질병 유전자' 깨운다..한국인서 첫 확인
5 : 이제 세상에 단 20마리..'판다 닮은 돌고래' 또 그물 걸려 숨져
w30====================================================


*2019년 03월 16일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : 

2019년 03월 20일의 뉴스 수집 중 ===================================
*2019년 03월 20일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : "버닝썬 '무삭제 성인물' 보세요".. 웃음거리 된 성범죄
2 : "한국인 싫다" 공항 난동 일본인, 알고 보니 후생성 간부
3 : 여학생 위로하는 저신다 아던 뉴질랜드 총리
4 : [썸_레터] "쉽게 안 잡힌다" 정준영 몰카보다 더 무서운 사실
5 : 애나는 인정하고 승리는 부인한 '마약 투약 혐의'
w20=======================================================


*2019년 03월 20일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : "미세먼지에 세워두나"..험난했던 50번째 삼성전자 주총
2 : 윤소하 원내대표 연설중 퇴장하는 자유한국당
3 : '경쟁하듯 더 크고 호화스럽게' 하늘로만 치솟는 교회들
4 : 사우디, 사막에 세계 최대 공원 조성..여의도공원 60배(종합)
5 : '희대의 연쇄살인마' 잭 더 리퍼는 누구?..DNA 분석으로 밝혀져
m20====================================================


*2019년 03월 20일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : "아파서, 임신해서, 가난해져서 죄송합니다"
2 : 나폴레옹·성심당 등 유명 제과업체·음식점 20곳 위생법 위반
3 : 연극공연 도중 "대한독립만세".. 관객 수백명도 목숨 걸고 "만세"
4 : 여기가 '여성안심지킴이집'이라고요? [이슈+]
5 : 이희진 부모살해 피의자, 범행후 이씨 동생 만나
w30====================================================


*2019년 03월 20일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : [단독] 美 카지노 마케터 입국 첩보..승리에 돈 받으러 왔나
2 : 강경화 "文대통령 

2019년 03월 24일의 뉴스 수집 중 ===================================
*2019년 03월 24일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : 발렌시아 구단주 딸 "승리가 왜 내게 전화했는지.."
2 : 승리, 발렌시아 구단주 딸이랑 무슨 관계?.. "혼란스럽다"
3 : [단독] 또 대학 단톡방서 성희롱.. "경북대 간호학과서 수개월 지속"
4 : 비상구 여니 3m 낭떠러지.. 이번엔 청주 노래방 추락사고
5 : 린사모·삼합회·발렌시아 구단주 딸까지..어마어마한 버닝썬 해외 투자자들
w20=======================================================


*2019년 03월 24일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : 靑 "무기 지니고 경호 활동하는 것은 지극히 당연"
2 : 나경원 "포항 지진, 여야 문제 아냐..지역 경제 살려야"
3 : "섬뜩한 경호 수칙 위반".."이전 정부도 똑같이 했다"
4 : '32년의 한' KAL858 동체 재수색 추진
5 : 靑경호원 '기관총'에 하태경 "섬뜩"..靑 "교과서적 대응"(종합)
m20====================================================


*2019년 03월 24일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 수학 포기하는 '수포자', 초등 3학년 분수에서 판가름
2 : "쇼크로 쓰러져 죽을 뻔"..헬스장에 퍼진 '위험한 주사기'
3 : "게임 그만해" 꾸짖는 엄마 살해한 조현병 아들 징역 7년
4 : 성급한 벚꽃 개화 "축제 당겨야할 듯"
5 : [날씨] 내일 낮부터 평년기온 회복..수도권 미세먼지 '나쁨'
w30====================================================


*2019년 03월 24일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : '그것이 알고싶다' 버닝썬 폭로 "대만 린사

2019년 03월 28일의 뉴스 수집 중 ===================================
*2019년 03월 28일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : '백인男 땀냄새 아시아女에 봄내음' 獨광고..성적대상화 비판
2 : 윤균상 고양이 갑자기 세상 떠났다..원인 '헤어볼' 뭐길래
3 : 고준희, "뉴욕 여배우·승리 비밀 누나" 의혹에 "아니다" 직접 해명
4 : 승강기 교체 중 17층서 갑자기 추락..작업자 2명 사망
5 : 82세에 휴대폰 게임 개발.. "디저트 같은 노년을 즐기세요"
w20=======================================================


*2019년 03월 28일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : 세월호 유가족 "DVR 바꿔치기 경악..상상도 못한일"
2 : 논란 됐던 '전범기업 인식표' 조례 상정 보류한다
3 : '손잡이 고무패킹이 다르다'
4 : 퇴위하는 아키히토 일왕에게.."저는 위안부라 불렸던 길원옥입니다" [정리뉴스]
5 : 500억 들여 건진 바다쓰레기, 육지의 '쓰레기山' 됐다
m20====================================================


*2019년 03월 28일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 2012년 KT에 아들·딸 '꽂아 넣은' 분들, 지금 떨고 계세요?
2 : 中 남성, 부부싸움후 홧김에 23층 아래로 가구·창문 던져
3 : "인건비 올라서.." 김밥말기도 자동화 바람
4 : [단독]"버닝썬 물뽕 의심반응 테스트기, 경찰이 그냥 버려"
5 : '기초학력 미달' 치솟는데 당국 수수방관..진단도 대책도 '미달'
w30====================================================


*2019년 03월 28일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : "버닝썬 물뽕 영상이래"..정준영 욕하며 

2019년 04월 02일의 뉴스 수집 중 ===================================
*2019년 04월 02일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 ====
1 : [끝까지판다] '영업 중단' 버닝썬, 수상한 밤샘 작업..자료 파기 포착
2 : [단독]'정준영 불법촬영 대화방' 있던 또다른 유명 연예인 김모씨 곧 경찰 조사.."조사 뒤 입건 검토"
3 : "50원 비닐 대신 800원 장바구니 구매하라고?" 비닐 단속 현장
4 : [단독] 황하나 "경찰청장, 우리 아빠랑 '베프'야"
5 : [단독] 국립대 안에 떡하니 '일본군 위안소'..태국 공문서 확인
w20=======================================================


*2019년 04월 02일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 ====
1 : [안녕? 자연] 머리에 공기방울 쓴채 최대 16분 잠수..신비한 도마뱀 발견
2 : [현장르포]"얼마까지 보셨어요?"..휴대폰 불법 보조금 여전
3 : 육사 '연좌제 얼차려' 논란..4명 음주했다고 900명 야간구보
4 : [르포] 유권자만 9억명..'세계 최대 민주주의 선거' 印 총선 현장
5 : 황교안 축구장 유세 처벌?..선관위 "경미한 사안..행정조치"
m20====================================================


*2019년 04월 02일의 뉴스 중 30대 여성이 많이 본 기사입니다 ====
1 : 재벌가 홀린 마약.. 황하나 이어 SK·현대가 자녀들도 마약 혐의
2 : [단독]"택시에 두고 내린 가방에 마약이"..제발로 경찰서 간 30대
3 : 태연한 황하나, '마약김치' 홍보까지..남양유업은 선긋기(종합)
4 : [단독] 양산실종아동 엄마, 슬리퍼만 신고 눈물 펑펑
5 : "트라우마..아기가 자기 뺨을 때려요"
w30====================================================


*2019년 